# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [40]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data() # get data
print(x.shape, y.shape)

(1000, 3) (1000,)


In [41]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    n=x.shape[0]
    data=np.concatenate([x,y.reshape(-1,1)], axis=1)
    np.random.shuffle(data)
    X_train, X_test=data[:int(n*train_size),:-1], data[int(n*train_size):,:-1]
    y_train, y_test=data[:int(n*train_size),-1], data[int(n*train_size):,-1]
    
    return X_train, X_test, y_train, y_test

In [42]:
X_train, X_test, y_train, y_test= split_data(x,y,0.8) # split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [43]:
def covariance(x, mu):
  mu=np.mean(x, axis=0)
  n,d=x.shape
  cov=np.zeros((d,d))
  print(cov.shape)
  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.
  for i in range(d):
    for j in range(d):
      cov[i,j]=np.sum((x[:,i]-mu[i])*(x[:,j]-mu[j]))/(len(x)-1)
  return cov

In [44]:
mu=np.mean(x, axis=0)
covariance(X_train,mu )

(3, 3)


array([[1.83174045, 0.04449634, 1.00795144],
       [0.04449634, 1.04155271, 0.05734296],
       [1.00795144, 0.05734296, 1.80997036]])

In [45]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu=None
    self.phi=None
    self.sigma=None
    
  def fit(self,x,y):
    k=  2 # Number of class.
    d= x.shape[1]  # input dim
    m= x.shape[0] # Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros(k)# d-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.
   
    for idx in range(k): 
      self.phi[idx]=np.sum(idx==y)/m
      self.mu[idx]=np.mean(x[idx==y], axis=0)
      self.sigma[idx]=covariance(x[idx==y], self.mu[idx])

    return self.phi, self.mu, self.sigma

  def predict_proba(self,x):
    
    # reshape or flatt x.
    #x= np.reshape(-1,1)
    d= x.shape[0]
    k_class=self.mu.shape[0] # Number of classes we have in our case it's k = 2
    proba=np.zeros((d,k_class))
    
    ## START THE LEARNING: estimate mu, phi and sigma.
    for i in range(k_class):
      sigma_det=np.linalg.det(self.sigma[i])
      sigma_inv=np.linalg.inv(self.sigma[i])
      for j in range(x.shape[0]):
        t=1/((2*np.pi)**(d/2)*(sigma_det**0.5))
        exp=-0.5*((x[j]-self.mu[i]).T)@(sigma_inv)@(x[j]-self.mu[i])
        t2=np.exp(exp)
        proba[j, i]= t*t2*self.phi[i]
    return proba

  def predict(self,x):
    pred=self.predict_proba(x)
    ypred=np.argmax(pred, axis=1)
    return ypred
  
  def accuracy(self, y, ypreds):
    reslt=np.mean(y==ypreds)
    return reslt*100

In [46]:
model= GDA()
model.fit(X_train,y_train)

(3, 3)
(3, 3)


(array([0.49, 0.51]), array([[ 0.98623346,  1.04799839,  0.99955429],
        [-1.01063579,  0.96511727, -0.96654476]]), array([[[ 0.86014661, -0.38710893, -0.06176344],
         [-0.38710893,  1.74880995,  0.10050906],
         [-0.06176344,  0.10050906,  0.04009008]],
 
        [[ 0.81097036,  0.37794805,  0.10960257],
         [ 0.37794805,  0.36128408, -0.06402819],
         [ 0.10960257, -0.06402819,  1.61594903]]]))

In [47]:
yproba= model.predict_proba(X_test)
yproba

array([[1.66415108e-080, 2.32988605e-081],
       [2.02653476e-081, 2.18328044e-101],
       [1.52154211e-082, 4.89726369e-081],
       [1.65320589e-125, 5.34797614e-081],
       [2.98157435e-080, 1.51750741e-081],
       [7.87000033e-106, 3.82463115e-081],
       [3.11528886e-102, 2.36884361e-081],
       [1.58786345e-080, 8.95560547e-082],
       [6.05960755e-081, 9.03310887e-086],
       [6.36434080e-081, 2.58044623e-082],
       [1.66340310e-080, 1.35825850e-089],
       [3.01334469e-080, 1.10325424e-082],
       [9.33293323e-151, 7.24185001e-081],
       [1.82887924e-081, 1.84708886e-084],
       [2.00800279e-111, 1.56778369e-080],
       [9.37552439e-087, 8.88948998e-081],
       [2.65102764e-082, 3.93778433e-100],
       [5.12005331e-081, 6.39532210e-089],
       [9.58595968e-081, 2.29429948e-081],
       [7.88191558e-093, 1.31314281e-080],
       [6.34058096e-126, 1.48388453e-080],
       [5.09242609e-081, 1.05518604e-081],
       [3.56361261e-144, 4.97003090e-081],
       [2.6

In [48]:
ypreds= model.predict(X_test)
ypreds

array([0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1])

In [49]:
model.accuracy(y_test, ypreds)

98.5

### Logistic Regression

In [55]:
class LogisticRegression:
  '''
  The goal of this class is to create a LogisticRegression class, 
  that we will use as our model to classify data point into a corresponding class
  '''
  def __init__(self,lr,n_epochs):
    self.lr = lr
    self.n_epochs = n_epochs
    self.train_losses = []
    self.w = None
    self.weight = []

  def add_ones(self, x):
    ##### WRITE YOUR CODE HERE #####
    one = np.ones((x.shape[0],1))
    return np.hstack((one,x))
    #### END CODE ####

  def sigmoid(self, x):
    return 1/(1+np.exp(-x@self.w))


  def cross_entropy(self, x, y_true):
    ##### WRITE YOUR CODE HERE #####
    y_pred = self.sigmoid(x)
    loss = -np.mean(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
    return loss
    #### END CODE ####
  
  def predict_proba(self,x):  #This function will use the sigmoid function to compute the probalities
    ##### WRITE YOUR CODE HERE #####
    x= self.add_ones(x)
    proba = self.sigmoid(x)
    return proba
    #### END CODE ####

  def predict(self,x):
    ##### WRITE YOUR CODE HERE #####
    probas = self.predict_proba(x)
    output = [0 if p<0.5 else 1 for p in probas]#np.where(probas>=0.5, 1, 0)      #convert the probalities into 0 and 1 by using a treshold=0.5
    return output
    #### END CODE ####

  def fit(self,x,y):
    # Add ones to x
    x=self.add_ones(x)

    # reshape y if needed
    y=y.reshape(-1,1)

    # Initialize w to zeros vector >>> (x.shape[1])
    self.w=np.zeros((x.shape[1],1))

    for epoch in range(self.n_epochs):
      # make predictions
      ypred = self.sigmoid(x)

      #compute the gradient
      dl = (-1/x.shape[0])*(x.T@(y-ypred))

      #update rule
      self.w=self.w-self.lr*dl

      #Compute and append the training loss in a list
      loss = self.cross_entropy(x,y)
      self.train_losses.append(loss)

      if epoch%1000 == 0:
        print(f'loss for epoch {epoch}  : {loss}')

  def accuracy(self,y_true, y_pred):
    ##### WRITE YOUR CODE HERE #####
    acc = np.mean(y_true==y_pred)*100
    return acc
    #### END CODE ####

In [56]:
model = LogisticRegression(0.01,n_epochs=10000)
model.fit(X_train,y_train)

loss for epoch 0  : 0.6882546546861045
loss for epoch 1000  : 0.19487474396451476
loss for epoch 2000  : 0.1708657974798757
loss for epoch 3000  : 0.16187807712262728
loss for epoch 4000  : 0.1570005916192161
loss for epoch 5000  : 0.15385948502076208
loss for epoch 6000  : 0.15163299851168463
loss for epoch 7000  : 0.1499592016785418
loss for epoch 8000  : 0.1486518169813554
loss for epoch 9000  : 0.1476036002821396


In [57]:
ypred_train = model.predict(X_train)
acc = model.accuracy(y_train,ypred_train)
print(f"The training accuracy is: {acc}")
print(" ")

ypred_test = model.predict(X_test)
acc = model.accuracy(y_test,ypred_test)
print(f"The test accuracy is: {acc}")

The training accuracy is: 95.75
 
The test accuracy is: 95.0
